# 神经网络与深度学习入门

本笔记本介绍了神经网络的基本结构、前向传播、使用NumPy实现推理，以及TensorFlow的基本使用。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 1. 神经网络基本结构

In [ ]:
print("神经网络基本结构:")
print("1. 输入层：接收原始特征数据")
print("2. 隐藏层：处理和转换特征")
print("3. 输出层：产生最终预测结果")

print("\n神经网络的数学表示:")
print("- 每个神经元：z = w·x + b，a = σ(z)")
print("- w：权重参数")
print("- b：偏置参数")
print("- σ：激活函数")
print("- a：神经元输出")

## 2. 激活函数

In [ ]:
def sigmoid(z):
    """
    sigmoid激活函数
    """
    return 1 / (1 + np.exp(-z))

def relu(z):
    """
    ReLU激活函数
    """
    return np.maximum(0, z)

def tanh(z):
    """
    tanh激活函数
    """
    return np.tanh(z)

# 可视化激活函数
z = np.linspace(-10, 10, 100)
plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
plt.plot(z, sigmoid(z))
plt.title('Sigmoid激活函数')
plt.grid(True)

plt.subplot(1, 3, 2)
plt.plot(z, relu(z))
plt.title('ReLU激活函数')
plt.grid(True)

plt.subplot(1, 3, 3)
plt.plot(z, tanh(z))
plt.title('tanh激活函数')
plt.grid(True)

plt.tight_layout()
plt.show()

## 3. 前向传播实现

In [ ]:
def initialize_parameters(layer_dims):
    """
    初始化神经网络参数
    
    参数:
    layer_dims: 包含各层神经元数量的列表
    
    返回:
    parameters: 包含权重和偏置的字典
    """
    np.random.seed(42)  # 设置随机种子以保证结果可重复
    parameters = {}
    L = len(layer_dims)
    
    for l in range(1, L):
        # 初始化权重，使用He初始化
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * np.sqrt(2 / layer_dims[l-1])
        # 初始化偏置为0
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
    
    return parameters

def forward_propagation(X, parameters):
    """
    神经网络前向传播
    
    参数:
    X: 输入特征 (n_features, m_samples)
    parameters: 包含权重和偏置的字典
    
    返回:
    AL: 输出层激活值
    caches: 包含中间值的字典，用于反向传播
    """
    caches = {}
    A = X
    L = len(parameters) // 2  # 网络层数
    
    # 前向传播通过隐藏层
    for l in range(1, L):
        W = parameters['W' + str(l)]
        b = parameters['b' + str(l)]
        
        # 线性变换
        Z = np.dot(W, A) + b
        # 应用ReLU激活函数
        A = relu(Z)
        
        # 保存中间值
        caches['Z' + str(l)] = Z
        caches['A' + str(l)] = A
    
    # 输出层
    WL = parameters['W' + str(L)]
    bL = parameters['b' + str(L)]
    ZL = np.dot(WL, A) + bL
    AL = sigmoid(ZL)  # 二分类问题使用sigmoid
    
    caches['Z' + str(L)] = ZL
    caches['A' + str(L)] = AL
    
    return AL, caches

## 4. 神经网络推理示例

In [ ]:
# 定义网络结构
layer_dims = [2, 4, 4, 1]  # 2个输入特征，2个隐藏层（各4个神经元），1个输出

# 初始化参数
parameters = initialize_parameters(layer_dims)

# 生成示例输入
X = np.array([[0.5, 0.2], [0.3, 0.8]]).T  # 2个样本，每个样本2个特征
print(f"输入形状: {X.shape}")

# 前向传播（推理）
AL, caches = forward_propagation(X, parameters)

print(f"输出预测: {AL.ravel()}")
print(f"预测类别: {(AL > 0.5).astype(int).ravel()}")

## 5. 使用NumPy实现简单神经网络

In [ ]:
class SimpleNeuralNetwork:
    """
    简单的三层神经网络实现
    """
    def __init__(self, input_dim, hidden_dim, output_dim):
        """
        初始化神经网络
        
        参数:
        input_dim: 输入特征维度
        hidden_dim: 隐藏层神经元数量
        output_dim: 输出层神经元数量
        """
        # 初始化参数
        self.W1 = np.random.randn(hidden_dim, input_dim) * np.sqrt(2 / input_dim)
        self.b1 = np.zeros((hidden_dim, 1))
        self.W2 = np.random.randn(output_dim, hidden_dim) * np.sqrt(2 / hidden_dim)
        self.b2 = np.zeros((output_dim, 1))
    
    def forward(self, X):
        """
        前向传播
        
        参数:
        X: 输入特征 (n_features, m_samples)
        
        返回:
        y_pred: 预测结果
        """
        # 隐藏层
        Z1 = np.dot(self.W1, X) + self.b1
        A1 = relu(Z1)
        
        # 输出层
        Z2 = np.dot(self.W2, A1) + self.b2
        y_pred = sigmoid(Z2)
        
        return y_pred
    
    def predict(self, X, threshold=0.5):
        """
        预测类别
        
        参数:
        X: 输入特征 (n_features, m_samples)
        threshold: 阈值
        
        返回:
        predictions: 预测类别
        """
        y_pred = self.forward(X)
        return (y_pred > threshold).astype(int)

# 创建神经网络实例
nn = SimpleNeuralNetwork(input_dim=2, hidden_dim=4, output_dim=1)

# 测试预测
X_test = np.array([[0.1, 0.9], [0.8, 0.2], [0.5, 0.5]]).T
predictions = nn.predict(X_test)
print(f"预测类别: {predictions.ravel()}")

## 6. TensorFlow 基础介绍

In [ ]:
try:
    import tensorflow as tf
    print(f"TensorFlow版本: {tf.__version__}")
    
    # 创建常量张量
    a = tf.constant(5.0)
    b = tf.constant(3.0)
    
    # 执行计算
    c = a * b
    print(f"TensorFlow计算: 5.0 * 3.0 = {c.numpy()}")
    
    # 创建变量
    W = tf.Variable(tf.random.normal(shape=(2, 3)))
    b = tf.Variable(tf.zeros(shape=(3,)))
    print(f"权重形状: {W.shape}")
    print(f"偏置形状: {b.shape}")
    
    # 使用tf.data创建数据集
    X = tf.random.normal(shape=(100, 2))
    y = tf.random.uniform(shape=(100, 1), minval=0, maxval=2, dtype=tf.int32)
    
    # 创建数据集
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.batch(32).shuffle(100)
    
    print("\nTensorFlow数据集示例:")
    for batch_X, batch_y in dataset.take(1):
        print(f"批处理X形状: {batch_X.shape}")
        print(f"批处理y形状: {batch_y.shape}")
        
except ImportError:
    print("TensorFlow未安装，跳过TensorFlow示例")
    print("请使用 'pip install tensorflow' 安装TensorFlow")

## 7. 深度学习入门总结

In [ ]:
print("神经网络基本概念:")
print("1. 神经网络是由多个神经元组成的层级结构")
print("2. 每个神经元执行线性变换后接非线性激活函数")
print("3. 前向传播是从输入到输出的计算过程")
print("4. 反向传播用于更新参数")

print("\n常用激活函数:")
print("1. Sigmoid: 用于二分类输出层")
print("2. ReLU: 用于隐藏层，解决梯度消失问题")
print("3. Tanh: 用于隐藏层，输出范围在[-1,1]")
print("4. Softmax: 用于多分类输出层")

print("\n深度学习框架:")
print("1. TensorFlow: Google开发，功能强大")
print("2. PyTorch: Facebook开发，动态计算图")
print("3. Keras: 高级API，可基于TensorFlow或其他后端")

print("\n深度学习应用:")
print("1. 计算机视觉: 图像分类、目标检测、图像分割")
print("2. 自然语言处理: 文本分类、机器翻译、情感分析")
print("3. 语音识别: 语音转文本、语音助手")
print("4. 推荐系统: 商品推荐、内容推荐")